In [1]:
from circuit import Circuit
import numpy as np
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

## Basic use

We initialize a circuit with a single quantum resource: a qubit. The "p" indicates that this qudit is a physical index into the unitary that this circuit represents. The "2" indicates that the qubit has 2 levels (of course, for a qubit, this is by definition; using another number will throw an error).

In [2]:
c = Circuit([("qubit", "p", 2)])

There are two qudit types to choose from: "qubit" and "cavity". There are 3 gates that you can add to a circuit: an arbitrary rotation on a qubit, an arbitrary displacement on a cavity, and an entangling SNAP gate between a qubit and a cavity. Our circuit only has a qubit, so we can only add a rotation; trying to add another type of gate will throw an error.

Typically, the add_gate method requires you to specify the index of the qudit that the gate acts on, but if the appropriate qudit can be uniquely inferred from the register, it's optional. Here, since there's only one qubit, it's unambiguous which qubit is being rotated, so I don't specify the qudit_ids.

In [3]:
c.add_gate("rotation")

If you want to specify the qudit_ids for a gate, the parameter to pass is called qids, a list of ints specifying all the qudits that the gate acts on. A rotation gate only acts on one qudit, so qids will be a length-1 list. Qudits are labelled by their index in the list that you passes into the Circuit constructor.

In [4]:
c.add_gate("rotation", qids=[0])

As soon as you've added all the gates you need to, make sure to assemble the circuit. You can't do anything with the circuit until you assemble it.

In [5]:
c.assemble()

An assembled circuit has some useful information that you can access. You can get the number of free parameters that this circuit needs directly from the circuit as shown below. Most of the circuit's metadata is stored in its RegisterInfo data structure. 

In [6]:
print("num params: {}".format(c.n_params))
print("total hilbert space dimension: {}".format(c.regInfo.dim))
print("qudit types: {}".format(c.regInfo.qudit_types))
print("qudit levels: {}".format(c.regInfo.qudit_levels))
print("tensor shape (p_in, b_in, p_out, b_out): {}".format(c.regInfo.tensor_shape))

num params: 6
total hilbert space dimension: 2
qudit types: ['qubit']
qudit levels: [2]
tensor shape (p_in, b_in, p_out, b_out): (2, 1, 2, 1)


There are 6 params since we added two rotations and each one takes three parameters: theta (the polar angle of the rotation axis), phi (azimuthal angle of rotation axis), and rotangle (the angle of rotation around the rotation axis). If we set theta and phi for each gate arbitrarily, but set rotangle=0 for both gates, clearly there is no rotation and we should get the identity matrix back. You get the unitary matrix by calling `circuit.evaluate(param_vector)`.

Note that the parameter vector is organized gate-by-gate. So the first rotation gate reads `params[0:3]` and the second gate reads `params[3:6]`.

In [7]:
params = np.array([2.13, -1.43, 0, 0.22, 5.44, 0])
unitary = c.evaluate(params)
print(unitary)

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]


## Parametrizer functions

Sometimes we don't want an arbitrary rotation; we'd instead like to parametrize a particular rotation, for example an arbitrary rotation around the Z axis. We can do this by providing additional arguments to the `add_gate` function: the number of free parameters in the gate, and a "parametrizer function" that specifies how the native parameters depend on the parameters the gate receives.

In this case, the arbitrary rotation around the Z axis is parametrized by one real number. The native parameters theta and phi are both zero, since the rotation axis is the Z axis. The rotangle parameter takes the value of the parameter passed into the gate. So, out parametrizer function should return `[0, 0, params[0]]`. I use a lambda function here since this is pretty trivial.

The parametrizer function takes as an argument its slice of the parameter vector (in this case, `p` is a vector slice of length 1, since `n_params=1`). It should output a list of the native parameters -- for a rotation gate, this list should have length 3.

In [8]:
c = Circuit([("qubit", "p", 2)])
c.add_gate("rotation", n_params=1, fn=lambda p: [0, 0, p[0]])
c.assemble()

The following code uses this Z-axis rotation, parametrized by `rotangle` $=\pi/2$. Acting on a Z-aligned qubit `[1, 0]`, it induces only a global phase, as expected. Acting on an X-aligned spinor, it produces a Y-aligned spinor (with a global phase).

In [9]:
print("pi/2 rotation around Z axis")
print()
params = np.array([np.pi/2])
unitary = c.evaluate(params)

print("effect on [1, 0]")
state = np.array([1, 0])
out = unitary @ state
# print without global phase
print(out/out[0])
print()

print("effect on [1, 1]/sqrt(2)")
state = np.array([1, 1]) / np.sqrt(2)
out = unitary @ state
# print without global phase
print(out/out[0])

pi/2 rotation around Z axis

effect on [1, 0]
[ 1.-0.j -0.+0.j]

effect on [1, 1]/sqrt(2)
[ 1.-0.j -0.+1.j]


## Two qudits

Here's another circuit, this time with two resources: a qubit and a cavity. Notice that the qubit is a physical index while the cavity represents a bond index (hence the "b"). The cavity has 10 levels, i.e. its local Hilbert space has dimension 10.

In [10]:
c = Circuit([("qubit", "p", 2),
             ("cavity", "b", 10)])
c.add_gate("displacement")
c.add_gate("snap")
c.add_gate("displacement")
c.assemble()

print("num params: {}".format(c.n_params))
print("total hilbert space dimension: {}".format(c.regInfo.dim))
print("qudit types: {}".format(c.regInfo.qudit_types))
print("qudit levels: {}".format(c.regInfo.qudit_levels))
print("tensor shape (p_in, b_in, p_out, b_out): {}".format(c.regInfo.tensor_shape))

num params: 14
total hilbert space dimension: 20
qudit types: ['qubit', 'cavity']
qudit levels: [2, 10]
tensor shape (p_in, b_in, p_out, b_out): (2, 10, 2, 10)


Here, I'm using the circuit and parameters specified in Heeres et al. "Cavity State Manipulation Using Photon-Number Selective Phase Gates" to produce the $|1\rangle$ Fock state in the cavity, starting from a spin-up qubit and ground state cavity.

In [11]:
params = np.array([1.14, 0]+[np.pi]+[0]*9+[-0.58, 0])
state = np.zeros(20)
state[0] = 1
out = c.evaluate(params) @ state
assert abs(np.linalg.norm(out) - 1) < 1e-5
assert np.abs(out[1]) > 0.99
print("Successfully created Fock state |1>")

Successfully created Fock state |1>


## Some more sample code and tests

Here, I show that SNAP gates and displacement gates do not produce entanglement, given that the qubit is initialized as Z-aligned. I do this by showing that the final unitary is block diagonal (so any spin-up qubit remains spin-up regardless of the circuit, and likewise for spin-down).

Note that we can get the 4-leg tensor representation of the unitary, with shape (phys, bond, phys, bond). This test also shows that the tensor reshaping is done correctly.

In [12]:
c = Circuit([("qubit", "p", 2),
             ("cavity", "b", 10)])
c.add_gate("displacement")
c.add_gate("snap")
c.add_gate("displacement")
c.add_gate("snap")
c.assemble()

rng = np.random.default_rng()

params = rng.uniform(high=2*np.pi, size=c.n_params)
tensor = c.get_tensor(params)
assert (tensor[0, :, 1, :] == np.zeros((10, 10))).all()
assert (tensor[1, :, 0, :] == np.zeros((10, 10))).all()
print("Z-aligned qubits remain unentangled")

Z-aligned qubits remain unentangled


Here are some final tests to show that everything behaves nicely.

In [13]:
from scipy.linalg import expm

rng = np.random.default_rng()
paulix = np.array([[0, 1], [1, 0]])
pauliy = np.array([[0, -1j], [1j, 0]])

print("TEST: Cavity X-rotation")
c = Circuit([("cavity", 'p', 2)])
c.add_gate("displacement", n_params=1, fn=lambda p: [0, -p[0]/2])
c.assemble()
for i in range(10):
    # random parameter between 0 and 2pi
    theta = rng.uniform(high=2*np.pi)
    u = c.evaluate(np.array([theta]))
    # the correct gate for an X-rotation on a spinor
    correct = expm(-1j/2 * theta * paulix)
    # check that the matrices agree within 1e-5
    assert np.allclose(u, correct, atol=1e-5)
print("PASS\n")

# the correct gate for a Z-to-X rotation on a spinor
correct = expm(-1j/2 * np.pi/2 * pauliy)

print("TEST: Cavity Z-to-X rotation")
c = Circuit([("cavity", 'p', 2)])
# can show that a displacement of pi/4+0i is equivalent to a ZX rotation
c.add_gate("displacement", n_params=0, fn=lambda p: [np.pi/4, 0])
c.assemble()
# param vector is empty
u = c.evaluate(np.array([]))
assert np.allclose(u, correct, atol=1e-5)
print("PASS\n")

print("TEST: Qubit Z-to-X rotation")
c = Circuit([("qubit", 'p', 2)])
# pi/2 rotation around Y axis has theta, phi = pi/2, pi/2
c.add_gate("rotation", n_params=0, fn=lambda p: [np.pi/2, np.pi/2, np.pi/2])
c.assemble()
u = c.evaluate(np.array([]))
assert np.allclose(u, correct, atol=1e-5)
print("PASS\n")

TEST: Cavity X-rotation
PASS

TEST: Cavity Z-to-X rotation
PASS

TEST: Qubit Z-to-X rotation
PASS

